# Quantum ESPRESSO Workshop - Part 6: Band Structure Calculation

## Learning Objectives
1. Understand the band structure calculation workflow
2. Define high-symmetry k-paths for FCC lattice
3. Perform SCF → bands → post-processing workflow
4. Extract and visualize the band structure
5. Analyze the band gap

---

## 1. Band Structure Calculation Workflow

### The Three-Step Process

1. **SCF calculation** (`calculation = 'scf'`):
   - Self-consistent charge density
   - Dense k-point grid for accuracy
   - Saves charge density to `outdir`

2. **Non-SCF bands calculation** (`calculation = 'bands'`):
   - Uses converged charge density from SCF
   - Calculates eigenvalues along high-symmetry path
   - No self-consistency (just diagonalization)

3. **Post-processing** (`bands.x`):
   - Extracts eigenvalues in plottable format
   - Handles band crossings and symmetry

### Why Non-SCF?

- Band structure requires eigenvalues at **specific k-points** along symmetry lines
- These k-points are different from the uniform mesh used in SCF
- We don't need to redo SCF; the charge density is already converged

---

## 2. Setup

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import time
import json

# Paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Working directory
WORK_DIR = OUTPUT_DIR / '06_band_structure'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")

In [ ]:
# Load converged parameters
params_file = WORKSHOP_ROOT / 'converged_parameters.json'

if params_file.exists():
    with open(params_file, 'r') as f:
        params = json.load(f)
    ecutwfc = params.get('ecutwfc_recommended', 40.0)
    ecutrho_factor = params.get('ecutrho_factor', 8)
    kgrid = params.get('kpoints_recommended', 8)
    celldm1 = params.get('celldm1_optimized_bohr', 10.26)
    print("Loaded converged parameters:")
    print(f"  ecutwfc = {ecutwfc} Ry")
    print(f"  ecutrho = {ecutrho_factor} × ecutwfc = {ecutwfc * ecutrho_factor} Ry")
    print(f"  k-points = {kgrid}×{kgrid}×{kgrid}")
    print(f"  celldm(1) = {celldm1} Bohr")
else:
    ecutwfc = 40.0
    ecutrho_factor = 8
    kgrid = 8
    celldm1 = 10.26
    print("Using default parameters")

ecutrho = ecutwfc * ecutrho_factor

---

## 3. High-Symmetry K-Path for FCC

### FCC Brillouin Zone

The FCC Brillouin zone is a truncated octahedron. High-symmetry points:

| Point | Coordinates (2π/a) | Description |
|-------|-------------------|-------------|
| Γ | (0, 0, 0) | Zone center |
| X | (0, 1, 0) | Face center |
| L | (1/2, 1/2, 1/2) | Body diagonal |
| W | (1/2, 1, 0) | Edge center |
| K | (3/4, 3/4, 0) | |
| U | (1/4, 1, 1/4) | |

### Standard Path: Γ → X → W → K → Γ → L → U → W → L → K

In [ ]:
# Define high-symmetry points for FCC in crystal coordinates
# Note: QE uses crystal coordinates for K_POINTS {crystal_b}

HIGH_SYMMETRY_POINTS = {
    'Γ': (0.000, 0.000, 0.000),
    'X': (0.500, 0.000, 0.500),
    'W': (0.500, 0.250, 0.750),
    'K': (0.375, 0.375, 0.750),
    'L': (0.500, 0.500, 0.500),
    'U': (0.625, 0.250, 0.625),
}

# Define the k-path: Γ → X → W → K → Γ → L
# This is a common path that shows all important features
K_PATH = [
    ('Γ', 20),  # 20 points from Γ to X
    ('X', 10),  # 10 points from X to W
    ('W', 10),  # 10 points from W to K
    ('K', 20),  # 20 points from K to Γ
    ('Γ', 20),  # 20 points from Γ to L
    ('L', 0),   # End point
]

print("High-Symmetry K-Path for Silicon (FCC)")
print("=" * 50)
print(f"Path: {' → '.join([p[0] for p in K_PATH])}")
print(f"\nTotal k-points along path: {sum(p[1] for p in K_PATH)}")
print("\nHigh-symmetry points (crystal coordinates):")
for name, coords in HIGH_SYMMETRY_POINTS.items():
    print(f"  {name}: ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})")

---

## 4. Step 1: SCF Calculation

In [ ]:
# Generate SCF input file
scf_input = f"""&CONTROL
    calculation = 'scf'
    prefix = 'silicon'
    outdir = './tmp'
    pseudo_dir = '{PSEUDO_DIR}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
    nbnd = 12
/

&ELECTRONS
    conv_thr = 1.0e-8
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kgrid} {kgrid} {kgrid} 0 0 0
"""

# Write SCF input
scf_file = WORK_DIR / 'si_scf.in'
with open(scf_file, 'w') as f:
    f.write(scf_input)

print("SCF Input File:")
print("=" * 50)
print(scf_input)

In [ ]:
# Run SCF calculation
print("Running SCF calculation...")
start = time.time()

result = subprocess.run(
    ['pw.x', '-in', 'si_scf.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=300
)

elapsed = time.time() - start

# Save output
with open(WORK_DIR / 'si_scf.out', 'w') as f:
    f.write(result.stdout)

# Check convergence
if 'convergence has been achieved' in result.stdout:
    print(f"✓ SCF converged in {elapsed:.2f} s")
    
    # Extract Fermi energy
    for line in result.stdout.split('\n'):
        if 'Fermi energy' in line or 'highest occupied' in line:
            print(f"  {line.strip()}")
else:
    print("✗ SCF did not converge!")
    if result.stderr:
        print(f"Errors: {result.stderr[:500]}")

---

## 5. Step 2: Bands Calculation

In [ ]:
# Generate K_POINTS card for band structure
def generate_kpath_card(k_path, high_sym_points):
    """Generate K_POINTS {crystal_b} card for band structure."""
    lines = ["K_POINTS {crystal_b}"]
    lines.append(str(len(k_path)))
    
    for point_name, npts in k_path:
        coords = high_sym_points[point_name]
        lines.append(f"  {coords[0]:.6f} {coords[1]:.6f} {coords[2]:.6f} {npts}  ! {point_name}")
    
    return '\n'.join(lines)

kpath_card = generate_kpath_card(K_PATH, HIGH_SYMMETRY_POINTS)
print("K-path card:")
print(kpath_card)

In [ ]:
# Generate bands input file
bands_input = f"""&CONTROL
    calculation = 'bands'
    prefix = 'silicon'
    outdir = './tmp'
    pseudo_dir = '{PSEUDO_DIR}'
    verbosity = 'high'
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
    nbnd = 12
/

&ELECTRONS
    conv_thr = 1.0e-8
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

{kpath_card}
"""

# Write bands input
bands_file = WORK_DIR / 'si_bands.in'
with open(bands_file, 'w') as f:
    f.write(bands_input)

print("Bands calculation input written.")
print(f"Number of bands: 12 (8 valence + 4 conduction)")

In [ ]:
# Run bands calculation
print("Running bands calculation...")
start = time.time()

result = subprocess.run(
    ['pw.x', '-in', 'si_bands.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=300
)

elapsed = time.time() - start

# Save output
with open(WORK_DIR / 'si_bands.out', 'w') as f:
    f.write(result.stdout)

if 'JOB DONE' in result.stdout:
    print(f"✓ Bands calculation completed in {elapsed:.2f} s")
else:
    print("✗ Bands calculation may have failed")
    if result.stderr:
        print(f"Errors: {result.stderr[:500]}")

---

## 6. Step 3: Post-processing with bands.x

In [ ]:
# Generate bands.x input
bands_pp_input = f"""&BANDS
    prefix = 'silicon'
    outdir = './tmp'
    filband = 'si_bands.dat'
    lsym = .true.
/
"""

# Write bands.x input
bands_pp_file = WORK_DIR / 'si_bands_pp.in'
with open(bands_pp_file, 'w') as f:
    f.write(bands_pp_input)

print("bands.x input:")
print(bands_pp_input)

In [ ]:
# Run bands.x post-processing
print("Running bands.x post-processing...")

result = subprocess.run(
    ['bands.x', '-in', 'si_bands_pp.in'],
    capture_output=True,
    text=True,
    cwd=WORK_DIR,
    timeout=60
)

# Save output
with open(WORK_DIR / 'si_bands_pp.out', 'w') as f:
    f.write(result.stdout)

if (WORK_DIR / 'si_bands.dat').exists():
    print("✓ Band data file created: si_bands.dat")
else:
    print("✗ Band data file not found")
    print(f"Output: {result.stdout[:500]}")

# Check for gnu file (alternative format)
if (WORK_DIR / 'si_bands.dat.gnu').exists():
    print("✓ GNU plot file created: si_bands.dat.gnu")

---

## 7. Parse and Plot Band Structure

In [ ]:
def parse_bands_gnu(filename):
    """
    Parse bands.dat.gnu file from QE bands.x.
    
    Returns
    -------
    k_distances : array
        K-point distances along path
    bands : array
        Band energies, shape (nkpts, nbands)
    """
    data = []
    current_band = []
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line = new band
                if current_band:
                    data.append(current_band)
                    current_band = []
            else:
                parts = line.split()
                if len(parts) >= 2:
                    k = float(parts[0])
                    e = float(parts[1])
                    current_band.append((k, e))
    
    if current_band:
        data.append(current_band)
    
    # Convert to arrays
    if not data:
        return None, None
    
    k_distances = np.array([p[0] for p in data[0]])
    bands = np.array([[p[1] for p in band] for band in data]).T
    
    return k_distances, bands


def get_high_symmetry_positions(k_distances, k_path):
    """
    Calculate positions of high-symmetry points along k-path.
    """
    # High-symmetry points are at boundaries between segments
    positions = [0.0]
    labels = [k_path[0][0]]
    
    cumulative_pts = 0
    for i, (name, npts) in enumerate(k_path[:-1]):
        cumulative_pts += npts
        if cumulative_pts < len(k_distances):
            positions.append(k_distances[cumulative_pts])
        else:
            positions.append(k_distances[-1])
        labels.append(k_path[i+1][0])
    
    return positions, labels

In [ ]:
# Parse band structure data
gnu_file = WORK_DIR / 'si_bands.dat.gnu'

if gnu_file.exists():
    k_dist, bands = parse_bands_gnu(gnu_file)
    
    if k_dist is not None:
        print(f"Parsed band structure data:")
        print(f"  Number of k-points: {len(k_dist)}")
        print(f"  Number of bands: {bands.shape[1]}")
        print(f"  Energy range: {bands.min():.4f} to {bands.max():.4f} eV")
    else:
        print("Failed to parse band data")
else:
    print(f"Band data file not found: {gnu_file}")
    print("Checking if raw bands.dat exists...")
    
    # Alternative: parse the raw bands.dat file
    bands_dat = WORK_DIR / 'si_bands.dat'
    if bands_dat.exists():
        print(f"Found {bands_dat}")
        with open(bands_dat, 'r') as f:
            print("First 20 lines:")
            for i, line in enumerate(f):
                if i < 20:
                    print(line.rstrip())

In [ ]:
# Extract Fermi energy from SCF output
fermi_energy = None
vbm = None  # Valence band maximum
cbm = None  # Conduction band minimum

scf_output_file = WORK_DIR / 'si_scf.out'
if scf_output_file.exists():
    with open(scf_output_file, 'r') as f:
        for line in f:
            if 'Fermi energy' in line:
                match = re.search(r'Fermi energy is\s+([\d.+-]+)', line)
                if match:
                    fermi_energy = float(match.group(1))
            if 'highest occupied, lowest unoccupied' in line:
                match = re.search(r'([\d.+-]+)\s+eV\s+([\d.+-]+)', line)
                if match:
                    vbm = float(match.group(1))
                    cbm = float(match.group(2))

if vbm is not None and cbm is not None:
    band_gap = cbm - vbm
    print(f"Band edges from SCF:")
    print(f"  VBM (highest occupied): {vbm:.4f} eV")
    print(f"  CBM (lowest unoccupied): {cbm:.4f} eV")
    print(f"  Band gap: {band_gap:.4f} eV")
    fermi_energy = vbm  # Use VBM as reference for semiconductors
elif fermi_energy:
    print(f"Fermi energy: {fermi_energy:.4f} eV")

In [ ]:
# Plot band structure
if k_dist is not None and bands is not None:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Get high-symmetry positions
    hs_positions, hs_labels = get_high_symmetry_positions(k_dist, K_PATH)
    
    # Shift energies relative to VBM (or Fermi energy)
    e_ref = vbm if vbm else (fermi_energy if fermi_energy else 0)
    bands_shifted = bands - e_ref
    
    # Plot bands
    for i in range(bands.shape[1]):
        color = 'blue' if i < 4 else 'red'  # Valence (blue) vs conduction (red)
        ax.plot(k_dist, bands_shifted[:, i], color=color, linewidth=1.5)
    
    # Add high-symmetry lines
    for pos in hs_positions:
        ax.axvline(x=pos, color='gray', linestyle='--', alpha=0.5)
    
    # Add Fermi level / VBM
    ax.axhline(y=0, color='green', linestyle='--', linewidth=1.5, label='VBM')
    
    # Labels and formatting
    ax.set_xticks(hs_positions)
    ax.set_xticklabels(hs_labels, fontsize=14)
    ax.set_xlabel('Wave Vector', fontsize=14)
    ax.set_ylabel('Energy (eV)', fontsize=14)
    ax.set_title('Silicon Band Structure (PBE)', fontsize=16)
    ax.set_xlim(k_dist.min(), k_dist.max())
    ax.set_ylim(-12, 8)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add band gap annotation
    if vbm is not None and cbm is not None:
        gap = cbm - vbm
        ax.annotate(f'Band Gap = {gap:.2f} eV', 
                   xy=(0.98, 0.98), xycoords='axes fraction',
                   ha='right', va='top', fontsize=12,
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(str(WORK_DIR / 'silicon_bands.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nFigure saved to: {WORK_DIR / 'silicon_bands.png'}")
else:
    print("Cannot plot: band data not available")

---

## 8. Band Gap Analysis

In [ ]:
# Analyze band gap from band structure
if bands is not None:
    # Silicon has 8 valence electrons → 4 valence bands
    n_valence_bands = 4
    
    valence_bands = bands[:, :n_valence_bands]
    conduction_bands = bands[:, n_valence_bands:]
    
    vbm_from_bands = valence_bands.max()
    cbm_from_bands = conduction_bands.min()
    gap_from_bands = cbm_from_bands - vbm_from_bands
    
    # Find k-points for VBM and CBM
    vbm_idx = np.unravel_index(valence_bands.argmax(), valence_bands.shape)
    cbm_idx = np.unravel_index(conduction_bands.argmin(), conduction_bands.shape)
    
    print("Band Gap Analysis from Band Structure")
    print("=" * 50)
    print(f"VBM: {vbm_from_bands:.4f} eV at k-point index {vbm_idx[0]}")
    print(f"CBM: {cbm_from_bands:.4f} eV at k-point index {cbm_idx[0]}")
    print(f"\nBand gap: {gap_from_bands:.4f} eV")
    
    if vbm_idx[0] == cbm_idx[0]:
        print("Gap type: DIRECT")
    else:
        print("Gap type: INDIRECT")
    
    print("\n" + "=" * 50)
    print("Comparison with Experiment")
    print("-" * 50)
    print(f"{'Property':<25} {'DFT (PBE)':<15} {'Experiment'}")
    print(f"{'Band gap (eV)':<25} {gap_from_bands:<15.3f} 1.17 eV")
    print(f"{'Gap type':<25} {'Indirect':<15} Indirect")
    print("\nNote: DFT-PBE underestimates band gaps (no derivative discontinuity)")

---

## 9. Save Results

In [ ]:
# Save band structure data for later use
if k_dist is not None and bands is not None:
    band_data = {
        'k_distances': k_dist.tolist(),
        'bands_ev': bands.tolist(),
        'vbm_ev': float(vbm_from_bands) if 'vbm_from_bands' in dir() else None,
        'cbm_ev': float(cbm_from_bands) if 'cbm_from_bands' in dir() else None,
        'band_gap_ev': float(gap_from_bands) if 'gap_from_bands' in dir() else None,
        'high_symmetry_positions': hs_positions if 'hs_positions' in dir() else None,
        'high_symmetry_labels': hs_labels if 'hs_labels' in dir() else None
    }
    
    with open(WORK_DIR / 'band_structure_data.json', 'w') as f:
        json.dump(band_data, f, indent=2)
    
    print(f"Band structure data saved to: {WORK_DIR / 'band_structure_data.json'}")

---

## Summary

In this notebook, we have:

1. ✓ Learned the SCF → bands → post-processing workflow
2. ✓ Defined high-symmetry k-paths for FCC Silicon
3. ✓ Performed the complete band structure calculation
4. ✓ Extracted and visualized the band structure
5. ✓ Analyzed the band gap (direct vs indirect)

### Key Results for Silicon (PBE)

| Property | DFT (PBE) | Experiment |
|----------|-----------|------------|
| Band gap | ~0.5-0.7 eV | 1.17 eV |
| Gap type | Indirect | Indirect |
| VBM location | Γ point | Γ point |
| CBM location | Near X | Near X |

### Important Notes

1. **DFT underestimates band gaps**: This is a fundamental limitation of DFT (missing derivative discontinuity)
2. **Qualitative features are correct**: Gap type, band dispersion, effective masses
3. **For accurate gaps**: Use GW, hybrid functionals, or DFT+U

### Next Notebook
→ **07_DOS_Calculation.ipynb**: Calculate and visualize the Density of States